In [1]:
import os
import pandas as pd
from statsbombpy import sb
from datetime import datetime
import warnings

In [2]:
# Definiowanie stałych
DEFAULT_SEASON = '2015/2016'
TOP5_LEAGUES = ['Italy', 'England', 'Spain', 'Germany', 'France']

# Wersja dla notebooka - używa katalogu bieżącego
try:
    # Próba użycia __file__ (działa w skryptach Python)
    DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
except NameError:
    # Fallback dla notebooków Jupyter
    DATA_DIR = os.path.join(os.getcwd(), 'data')
    
warnings.filterwarnings("ignore", category=UserWarning, module="statsbombpy")

In [3]:
def collect_shots_data(league, save_path=DATA_DIR, season_name=DEFAULT_SEASON, save_files=True):
    """
    Pobiera i zapisuje dane o strzałach z wybranej ligi dla sezonu 2015/16.
    
    Parameters:
    -----------
    league : str lub list
        Nazwa ligi lub lista lig (np. 'Italy', 'England', 'Spain', 'Germany', 'France')
    save_path: str, optional
        Ścieżka zapisu docelowego pliku (domyślnie folder 'data' w bieżącym katalogu)
    season_name: str, optional
        Nazwa sezonu (default: '2015/2016')
    save_files: bool, optional
        Czy zapisywać pliki CSV (default: True)
        
    Returns:
    --------
    pd.DataFrame
        DataFrame ze wszystkimi strzałami
    """
    # Tworzenie katalogu, jeśli nie istnieje
    if save_files and not os.path.exists(save_path):
        os.makedirs(save_path)
        print(f"Utworzono katalog: {save_path}")
    
    # Obsługa pojedynczej ligi lub listy lig
    if isinstance(league, str):
        leagues_to_process = [league]
    else:
        leagues_to_process = league
    
    all_shots_data = []
    
    # Przetwarzanie każdej ligi osobno
    for current_league in leagues_to_process:
        print(f"\nRozpoczynanie pobierania danych z ligi: {current_league}")
        
        # Pobranie danych o lidze
        try:
            free_comps = sb.competitions()
            
            # Filtrowanie wybranej ligi
            league_data = free_comps[(free_comps['season_name']==season_name) & 
                               (free_comps['country_name']==current_league)]
            
            if league_data.empty:
                print(f"Nie znaleziono danych dla ligi {current_league} w sezonie {season_name}. Pomijam.")
                continue
            
            competitions = list(league_data['competition_id'])
            
            # Pobranie ID meczów
            season_id = league_data['season_id'].iloc[0]
            all_matches = pd.concat([sb.matches(competition_id=comp_id, season_id=season_id) 
                                  for comp_id in competitions])
            matches_id = list(all_matches['match_id'])
            print(f"Znaleziono {len(matches_id)} meczów do przeanalizowania")
        except Exception as e:
            print(f"Błąd podczas pobierania meczów dla ligi {current_league}: {str(e)}")
            continue
        
        # Pobieranie danych o strzałach
        shot_data = []
        
        for idx, match_id in enumerate(matches_id):
            try:
                print(f"Przetwarzam mecz {idx+1}/{len(matches_id)}", end='\r')
                
                events = sb.events(match_id=match_id)
                shots = events[events['type']=='Shot']
                
                # Dodawanie tylko niepustych ramek danych
                if not shots.empty:
                    shot_data.append(shots)
                                
            except Exception as e:
                print(f"\nBłąd przy meczu {match_id}: {str(e)}")
                continue
        
        if shot_data:
            # Łączenie danych z tej ligi
            print("\nŁączenie danych...")
            league_shots = pd.concat(shot_data)
            
            # Zapisywanie pliku dla tej ligi
            if save_files:
                # Tworzenie poprawnej nazwy pliku
                season_str = season_name.replace("/", "_")
                output_filename = os.path.join(save_path, f'all_shots_{current_league}_{season_str}.csv')
                league_shots.to_csv(output_filename, index=False)
                print(f"Zapisano dane do pliku: {output_filename}")
            
            # Dodanie do zbiorczych danych
            all_shots_data.append(league_shots)
    
    # Łączenie danych ze wszystkich lig, jeśli jest więcej niż jedna
    if len(all_shots_data) > 0:
        all_shots = pd.concat(all_shots_data)
        
        # Zapisywanie zbiorczego pliku, jeśli przetworzono więcej niż jedną ligę
        if save_files and len(leagues_to_process) > 1:
            season_str = season_name.replace("/", "_")
            output_filename = os.path.join(save_path, f'all_shots_combined_{season_str}.csv')
            all_shots.to_csv(output_filename, index=False)
            print(f"\nZapisano zbiorcze dane do pliku: {output_filename}")
        
        return all_shots
    else:
        print("Nie pobrano żadnych danych.")
        return None

# Kod wykonywany tylko przy bezpośrednim uruchomieniu skryptu
if __name__ == "__main__":
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
        print(f"Utworzono katalog danych: {DATA_DIR}")
    
    # Pobieranie danych dla TOP5 lig
    print(f"Rozpoczynam pobieranie danych dla lig: {', '.join(TOP5_LEAGUES)}")
    shots_df = collect_shots_data(league=TOP5_LEAGUES, save_path=DATA_DIR)
    
    if shots_df is not None:
        print(f"Pobrano dane o {len(shots_df)} strzałach z {len(TOP5_LEAGUES)} lig")

Rozpoczynam pobieranie danych dla lig: Italy, England, Spain, Germany, France

Rozpoczynanie pobierania danych z ligi: Italy
Znaleziono 380 meczów do przeanalizowania
Przetwarzam mecz 380/380
Łączenie danych...
Zapisano dane do pliku: C:\Users\Bartek\Desktop\portfolio\data\all_shots_Italy_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: England
Znaleziono 380 meczów do przeanalizowania
Przetwarzam mecz 380/380
Łączenie danych...
Zapisano dane do pliku: C:\Users\Bartek\Desktop\portfolio\data\all_shots_England_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: Spain
Znaleziono 380 meczów do przeanalizowania
Przetwarzam mecz 380/380
Łączenie danych...
Zapisano dane do pliku: C:\Users\Bartek\Desktop\portfolio\data\all_shots_Spain_2015_2016.csv

Rozpoczynanie pobierania danych z ligi: Germany
Znaleziono 306 meczów do przeanalizowania
Przetwarzam mecz 306/306
Łączenie danych...
Zapisano dane do pliku: C:\Users\Bartek\Desktop\portfolio\data\all_shots_Germany_2015_2016.csv

Rozpoczyn